In [ ]:
# Plot options
import matplotlib.pyplot as plt

# Import required libraries
import numpy as np
import pyqtgraph as pg

# Import karabo libraries
import karabo_bridge as kb
import karabo_data as kd
import xfelmay2019 as tofoff


# Find brightest tof spectrum

In [ ]:
path='/gpfs/exfel/exp/SQS/201921/p002430/raw'
run = tofoff.runFormat( 232 )
runData = kd.RunDirectory(path+run)
tofavg = tofoff.tofAverager( runData )

In [ ]:
tofmax = None
maxsum = -1
tidmax = None
for tid, data in runData.trains():
    tofdata = data['SQS_DIGITIZER_UTC1/ADC/1:network']['digitizers.channel_1_A.raw.samples'][260000:280000]
    currsum = np.sum( np.abs( tofdata ) )
    if tofmax is None:
        tofmax = tofdata
        maxsum = np.sum( np.abs( tofmax ) )
        tidmax = tid
    elif currsum > maxsum:
        tofmax = tofdata
        maxsum = np.sum( np.abs( tofmax ) )
        tidmax = tid
        
print('Max TID is %d' % tidmax)


## Plot charge states 2->???

In [ ]:
%matplotlib inline 
plt.figure(figsize = (30,20))

z  = tofoff.normalizedTOF( tofavg, downsampleRange=(260000,280000) )
zraw = np.copy(z)
thresh=50.
z[ z<thresh ] = 0.
zf,zguess = findTOFPeaks( z )

zf = zf[zguess > thresh]
zguess = zguess[zguess>thresh]
    

plt.plot(z)
plt.plot(zf, zguess, 'o', ms=10, color='red')

# Use peaks to find fit

In [ ]:
# zpeaks = zf[ (zf>3500) & (zf<8000) ]
zpeaks = zf
print(zpeaks)

In [ ]:
zpeaks.size

In [ ]:
x = np.flip(zpeaks)
# m = 130.
m = 40.
y = m/np.arange(2,zf.size+2).astype(float)

polyCoef = np.polyfit(x,y,2)

fitPoly = np.poly1d(polyCoef)

plt.figure(figsize = (30,20))
plt.plot( x,y, 'b' )
plt.plot( x,fitPoly(x), 'r' )

In [ ]:
fitPoly

## Plot calibrated TOF spectrum

In [ ]:
xs = np.arange( zraw.size )
mperq = fitPoly(xs)
minAt = np.argmin(mperq)

mperq = mperq[minAt:]
zcalib = zraw[minAt:]
    
chargeNumber = 1./( mperq / m )

chargeCutoff = 20.
plt.figure(figsize = (30,20))
plt.plot( chargeNumber[chargeNumber < chargeCutoff], zcalib[chargeNumber < chargeCutoff] )
plt.xticks(np.arange(chargeCutoff+1.));

# Plot calibrated TOF for 10% beam on atomic xenon

In [ ]:
path='/gpfs/exfel/exp/SQS/201921/p002430/raw'
run = tofoff.runFormat( 221 )
runData = kd.RunDirectory(path+run)
tofavg = tofoff.tofAverager( runData )

In [ ]:
from scipy.signal import find_peaks_cwt
def findTOFPeaks( toftrace ):
    '''
    input:
        toftrace: use normalized tof trace generated from normalizedTOF, numpy array of floats
    output:
        peak_positions: returns indexes of the TOF peaks
        peak_values: returns height of TOF peaks
    Watthew Ware, 2019
    '''
    toftrace = np.abs(toftrace)
    base_width = 20.
    # Find indexes of inds
    zf = find_peaks_cwt(toftrace, [base_width])
    
    # Create averaging zones around peaks
    zguess = np.zeros_like(zf).astype(float) # allocate space

    for ii,zfi in enumerate(zf):
        xlow = (np.round( zfi - base_width/2. )).astype(int)
        if xlow < 0:
            xlow = 0
        xhigh = (np.round( zfi + base_width/2. )).astype(int)
        if xhigh > toftrace.size:
            xhigh = toftrace.size
            
        zguess[ii] = np.max(toftrace[xlow:xhigh])
        
    return zf, zguess

In [ ]:
%matplotlib inline 
plt.figure(figsize = (30,20))

z  = tofoff.normalizedTOF( tofavg, downsampleRange=(260000,280000) )
zraw = np.copy(z)
# thresh=0.
# z[ z<thresh ] = 0.
# zf,zguess = findTOFPeaks( z )

# zf = zf[zguess > thresh]
# zguess = zguess[zguess>thresh]

xs = np.arange( zraw.size )
mperq = fitPoly(xs)
minAt = np.argmin(mperq)

mperq = mperq[minAt:]
zcalib = zraw[minAt:]
    
mXe = 130
chargeNumber = 1./( mperq / mXe )
    

chargeCutoff = 50.
plt.figure(figsize = (30,20))
plt.plot( chargeNumber[chargeNumber < chargeCutoff], zcalib[chargeNumber < chargeCutoff] )
plt.xticks(np.arange(chargeCutoff+1.));
# plt.plot(zf, zguess, 'o', ms=10, color='red')

# Plot calibrated TOF for 30% beam on atomic xenon

In [ ]:
path='/gpfs/exfel/exp/SQS/201921/p002430/raw'
run = tofoff.runFormat( 222 )
runData = kd.RunDirectory(path+run)
tofavg = tofoff.tofAverager( runData )

In [ ]:
from scipy.signal import find_peaks_cwt
def findTOFPeaks( toftrace ):
    '''
    input:
        toftrace: use normalized tof trace generated from normalizedTOF, numpy array of floats
    output:
        peak_positions: returns indexes of the TOF peaks
        peak_values: returns height of TOF peaks
    Watthew Ware, 2019
    '''
    toftrace = np.abs(toftrace)
    base_width = 20.
    # Find indexes of inds
    zf = find_peaks_cwt(toftrace, [base_width])
    
    # Create averaging zones around peaks
    zguess = np.zeros_like(zf).astype(float) # allocate space

    for ii,zfi in enumerate(zf):
        xlow = (np.round( zfi - base_width/2. )).astype(int)
        if xlow < 0:
            xlow = 0
        xhigh = (np.round( zfi + base_width/2. )).astype(int)
        if xhigh > toftrace.size:
            xhigh = toftrace.size
            
        zguess[ii] = np.max(toftrace[xlow:xhigh])
        
    return zf, zguess

In [ ]:
%matplotlib inline 
plt.figure(figsize = (30,20))

z  = tofoff.normalizedTOF( tofavg, downsampleRange=(260000,280000) )
zraw = np.copy(z)
# thresh=0.
# z[ z<thresh ] = 0.
# zf,zguess = findTOFPeaks( z )

# zf = zf[zguess > thresh]
# zguess = zguess[zguess>thresh]

xs = np.arange( zraw.size )
mperq = fitPoly(xs)
minAt = np.argmin(mperq)

mperq = mperq[minAt:]
zcalib = zraw[minAt:]
    
mXe = 130
chargeNumber = 1./( mperq / mXe )
    

chargeCutoff = 50.
plt.figure(figsize = (30,20))
plt.plot( chargeNumber[chargeNumber < chargeCutoff], zcalib[chargeNumber < chargeCutoff] )
plt.xticks(np.arange(chargeCutoff+1.));
# plt.plot(zf, zguess, 'o', ms=10, color='red')

# Plot calibrated TOF for ~100% beam on atomic xenon

In [ ]:
path='/gpfs/exfel/exp/SQS/201921/p002430/raw'
run = tofoff.runFormat( 223 )
runData = kd.RunDirectory(path+run)
tofavg = tofoff.tofAverager( runData )

In [ ]:
from scipy.signal import find_peaks_cwt
def findTOFPeaks( toftrace ):
    '''
    input:
        toftrace: use normalized tof trace generated from normalizedTOF, numpy array of floats
    output:
        peak_positions: returns indexes of the TOF peaks
        peak_values: returns height of TOF peaks
    Watthew Ware, 2019
    '''
    toftrace = np.abs(toftrace)
    base_width = 20.
    # Find indexes of inds
    zf = find_peaks_cwt(toftrace, [base_width])
    
    # Create averaging zones around peaks
    zguess = np.zeros_like(zf).astype(float) # allocate space

    for ii,zfi in enumerate(zf):
        xlow = (np.round( zfi - base_width/2. )).astype(int)
        if xlow < 0:
            xlow = 0
        xhigh = (np.round( zfi + base_width/2. )).astype(int)
        if xhigh > toftrace.size:
            xhigh = toftrace.size
            
        zguess[ii] = np.max(toftrace[xlow:xhigh])
        
    return zf, zguess

In [ ]:
%matplotlib inline 
plt.figure(figsize = (30,20))

z  = tofoff.normalizedTOF( tofavg, downsampleRange=(260000,280000) )
zraw = np.copy(z)
# thresh=0.
# z[ z<thresh ] = 0.
# zf,zguess = findTOFPeaks( z )

# zf = zf[zguess > thresh]
# zguess = zguess[zguess>thresh]

xs = np.arange( zraw.size )
mperq = fitPoly(xs)
minAt = np.argmin(mperq)

mperq = mperq[minAt:]
zcalib = zraw[minAt:]
    
mXe = 130
chargeNumber = 1./( mperq / mXe )
    

chargeCutoff = 50.
plt.figure(figsize = (30,20))
plt.plot( chargeNumber[chargeNumber < chargeCutoff], zcalib[chargeNumber < chargeCutoff] )
plt.xticks(np.arange(chargeCutoff+1.));
# plt.plot(zf, zguess, 'o', ms=10, color='red')

# Plot calibrated TOF for 100% beam on atomic xenon

In [ ]:
path='/gpfs/exfel/exp/SQS/201921/p002430/raw'
run = tofoff.runFormat( 225 )
runData = kd.RunDirectory(path+run)
tofavg = tofoff.tofAverager( runData )

In [ ]:
from scipy.signal import find_peaks_cwt
def findTOFPeaks( toftrace ):
    '''
    input:
        toftrace: use normalized tof trace generated from normalizedTOF, numpy array of floats
    output:
        peak_positions: returns indexes of the TOF peaks
        peak_values: returns height of TOF peaks
    Watthew Ware, 2019
    '''
    toftrace = np.abs(toftrace)
    base_width = 20.
    # Find indexes of inds
    zf = find_peaks_cwt(toftrace, [base_width])
    
    # Create averaging zones around peaks
    zguess = np.zeros_like(zf).astype(float) # allocate space

    for ii,zfi in enumerate(zf):
        xlow = (np.round( zfi - base_width/2. )).astype(int)
        if xlow < 0:
            xlow = 0
        xhigh = (np.round( zfi + base_width/2. )).astype(int)
        if xhigh > toftrace.size:
            xhigh = toftrace.size
            
        zguess[ii] = np.max(toftrace[xlow:xhigh])
        
    return zf, zguess

In [ ]:
%matplotlib inline 
plt.figure(figsize = (30,20))

z  = tofoff.normalizedTOF( tofavg, downsampleRange=(260000,280000) )
zraw = np.copy(z)
# thresh=0.
# z[ z<thresh ] = 0.
# zf,zguess = findTOFPeaks( z )

# zf = zf[zguess > thresh]
# zguess = zguess[zguess>thresh]

xs = np.arange( zraw.size )
mperq = fitPoly(xs)
minAt = np.argmin(mperq)

mperq = mperq[minAt:]
zcalib = zraw[minAt:]
    
mXe = 130
chargeNumber = 1./( mperq / mXe )
    

chargeCutoff = 50.
plt.figure(figsize = (30,20))
plt.plot( chargeNumber[chargeNumber < chargeCutoff], zcalib[chargeNumber < chargeCutoff] )
plt.xticks(np.arange(chargeCutoff+1.));
# plt.plot(zf, zguess, 'o', ms=10, color='red')

plt.savefig('xenon-run225-fullbeam.png',)